# Image Classifier Fundamentals

To establish fundamentals, first a set of utilities necessary for image preprocessing and loading from the disk. Then we will use the k-Nearest Neighbor (k-NN) classifier to recognize image class labels using only the raw pixel intensities.

## Dataset: "Animals"

The "Animals" dataset is a simple dataset pieced together by Adrian Rosebrock to reveal the process of training an image classifier using different machine learning techniques. The dataset consists of three class labels, (1) dogs, (2) cats, and (3) pandas. Each class label has exactly 1,000 images, totaling in 3,000 images. The images within the dataset are actually subsets of both the Kaggle "[Dogs vs. Cats](https://www.kaggle.com/c/dogs-vs-cats)" challenge and the [ImageNet](https://www.image-net.org/) dataset. 

## Utilities
**Image Preprocessor**

Images are required to be preprocessed and scaled to have identical widths and heights suitable for machine learning algorithms. Though image preprocessing can take many forms in resizing, scaling, or respect of aspect ratio, this image preprocessor will only resize images and ignore aspect ratio.

In [2]:
# import necessary packages
import cv2

class SimplePreprocessor:
    def __init__(self, width, height, inter = cv2.INTER_AREA):
        # store the target image width, height, and interpolation
        # method used when resizing
        self.width = width
        self.height = height
        self.inter = inter
        
    def preprocess(self, image):
        # resize the image to a fixed size, ignoring the aspect ratio
        return cv2.resize(image, (self.width, self.height), interpolation = self.inter)

The imported packaged needed for our image preprocessor is cv2, otherwise known as OpenCV.

Following library imports, the constructor is seen with three arguments.
- *width*: Required, the desired width of the input image after resizing.
- *height*: Required, the desired height of our input image after resizing. 
- *inter*: Optional, choice of interpolation algorithm when resizing.

The *preprocess* function accepts a single input ad returns the desired processed image. 

**Image Loader**

In [4]:
# import the necessary packages packages
import numpy as np
import cv2
import os

class SimpleDatasetLoader:
    def __init__(self, preprocessors = None):
        # store the image preprocessor
        self.preprocessors = preprocessors
        
        # if the preprocessors are None, initialize them as an empty list
        if self.preprocessors is None: 
            self.preprocessors = []
        
    def load(self, imagePaths, verbose = -1):
        # initialize the list of features and labels
        data = []
        labels = []
        
        # loop over the input images
        for (i, imagePath) in enumerate(imagePaths):
            # load the image and extract the class label assuming
            # that our path has the following format:
            # /path/to/dataset/{class}/{image}.jpg
            image = cv2.imread(imagePath)
            label = imagePath.split(os.path.sep)[-2]
            
            # check to see if our preprocessors are not None
            if self.preprocessors is not None:
                # loop over preprocessors and apply each to the image
                for p in self.preprocessors:
                    image = p.preprocess(image)
                    
            # treat our processed image as a 'feature vector'
            # by updating the data list followed by the labels
            data.append(image)
            labels.append(label)
            
            if verbose > 0 and i > 0 and (i + 1) % verbose == 0:
                print("[INFO] processed {}/{}".format(i + 1, len(imagePaths)))
                
        # return a tuple of the data and labels
        return (np.array(data), np.array(labels))

The following library packages are imported:
- *numpy* - for numerical processing
- *cv2* - for OpenCV bindings
- *os* - to extract names of subdirectories in image paths

The constructor has one optional argument, *preprocessors*, in which a list of desired preprocessors can be inputed to process images accordingly. The input type being a list is important to being able to perform sequential modifications, such as resizing an image first and then scaling the resized image. 

The *load* function requires one argument, *imagePaths*, referring to the location of the dataset within the disk. Verbose refers to viewing the active updates taking place while the function is performing its objective. The *data* and *labels* variables begin an empty list in which to store the data we will iterate through given that *preprocessors* is not equal to *None*. 

## k-Nearest Neighbors (k-NN)

The k-NN algorithm is simple enough in which no 'learning' is actually taking place. Rather, k-NN is dependent on the distance feature vectors (RGB pixel intensities). That is, the algorithm classifies data points by finding the most common class among *k* closest 'points'. Whichever label has the most data points closest to the image of interest is considered the most likely label. The underlying assumption is images with the same class label will share similar characteristics between different images (i.e., data points), and this will be seen when expressed as a feature vector. 

We will use four steps to train a k-NN algorithm to classify the *animals* dataset based on the raw pixel intensities of images. 

1. Gather Dataset - Each image of the 'animals' dataset will be preprocessed using our SimplePreprocessor.
2. Split Dataset - The dataset will be split into a training and testing set. 
3. Train Classifier - The k-NN classifier will be trained on the training dataset. 
4. Evaluate - The k-NNs performance will be evaluated on the test set. 

In [9]:
# import packages
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

from imutils import paths
import argparse

# construct the argument parse and parse the arguments 
ap = argparse.ArgumentParser()
ap.add_argument("-d", "--dataset", required = True, help = "path to input dataset")
ap.add_argument("-k", "--neighbors", type = int, default = 1, help = "# of nearest neighbors for classification")
ap.add_argument("-j", "--jobs", type = int, default = -1, help = "# of jobs for k-NN distance (-1 uses all available cores)")
args = vars(ap.parse_args())

# grab the list of images we will be describing 
print("[INFO] loading images...")
imagePaths = list(paths.list_images["dataset"])

# initialize the image preprocessor, load the dataset from disk, 
# and reshape the data matrix
sp = SimplePreprocessor(32,32)
sdl = SimpleDatasetLoader(preprocessors=[sp])
(data, labels) = sdl.load(imagePaths, verbose = 500)
data = data.reshape((data.shape[0], 3072))

# show some information on memory consumption of the images
print("[INFO] features matrix: {:1f}MB".format(data.nbytes / (1024 * 1024.0)))

# encode the labels as integers 
le = LabelEncoder()
labels= le.fit_transform(labels)

# partition the data into training and testing splits using 75% of the data
# for training and the remaining 25% for testing
print("[INFO] evaluating a k-NN classifier...")
model = KNeighborsClassifier(n_neighbors = args['neighbors'],
                             n_jobs = args["jobs"])
model.fit(trainX,trainY)
print(classificationreport(testY, model.predict(testX),
                          target_names = le.classes_))

usage: ipykernel_launcher.py [-h] -d DATASET [-k NEIGHBORS] [-j JOBS]
ipykernel_launcher.py: error: the following arguments are required: -d/--dataset


SystemExit: 2

C:\Users\chris\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3452: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
